In [ ]:
!git clone https://github.com/arunponnusamy/object-detection-opencv

Cloning into 'object-detection-opencv'...
remote: Enumerating objects: 61, done.
remote: Total 61 (delta 0), reused 0 (delta 0), pack-reused 61
Unpacking objects: 100% (61/61), done.


In [ ]:
!wget https://pjreddie.com/media/files/yolov3.weights

--2021-11-26 21:00:28--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘yolov3.weights’

yolov3.weights      100%[===================>] 236.52M  20.8MB/s    in 12s     

2021-11-26 21:00:41 (19.4 MB/s) - ‘yolov3.weights’ saved [248007048/248007048]



In [ ]:
pip install opencv-python

In [ ]:
import cv2
import numpy as np
import os
from google.colab.patches import cv2_imshow

In [ ]:
cap = cv2.VideoCapture('input.mp4')
cap.set(3,480)
cap.set(4,640)

False

In [ ]:
classesf = "/content/object-detection-opencv/yolov3.txt"
config = "/content/object-detection-opencv/yolov3.cfg"
weights = "/content/yolov3.weights"
FILE_OUTPUT = "output.mp4"
classes = None
red_color = [190,196,255]
previous =[]


In [ ]:
# Checks and deletes the output file
# You cant have a existing file or it will through an error
if os.path.isfile(FILE_OUTPUT):
    os.remove(FILE_OUTPUT)

In [ ]:
def get_output_layers(net):
    
    layer_names = net.getLayerNames()
    
    output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]

    return output_layers

In [ ]:
# draw the bounding boxes
def draw_prediction(img, x, y, x_plus_w, y_plus_h, color):

    cv2.rectangle(img, (x,y), (x_plus_w,y_plus_h), color, 2)


In [ ]:
# moving_detection
def moving_detection(box_list,previous_list,t):
  for p in previous_list:
    if box_list[0]-t < p[0] < box_list[0]+t and box_list[1]-t < p[1] < box_list[1]+t and box_list[2]-t < p[2] < box_list[2]+t and box_list[3]-t < p[3] < box_list[3]+t:
      return True
  return False


In [ ]:
def video_read():
  global previous
  while True:
    ret , image = cap.read()
    if image is None:
      break
    if image[0,0,1] < image[0,0,2]:
      print(image[0,0,1] , image[0,0,2])
      Width = image.shape[1]
      Height = image.shape[0]
      blob = cv2.dnn.blobFromImage(image, 1.0/255, (416,416), (0,0,0), True, crop=False)
      net.setInput(blob)
      outs = net.forward(get_output_layers(net))
      class_ids = []
      confidences = []
      boxes = []
      conf_threshold = 0.5
      nms_threshold = 0.4
      for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5 and (classes[class_id] == "car" or classes[class_id] == "truck"):
                center_x = int(detection[0] * Width)
                center_y = int(detection[1] * Height)
                w = int(detection[2] * Width)
                h = int(detection[3] * Height)
                x = center_x - w / 2
                y = center_y - h / 2
                class_ids.append(class_id)
                confidences.append(float(confidence))
                boxes.append([x, y, w, h])
      indices = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold)

      for i in indices:
          i = i[0]
          box = boxes[i]
          x = box[0]
          y = box[1]
          w = box[2]
          h = box[3]
          print(box)
          if w * h > 1600: #the area of a rectangle > 1600
            if moving_detection(box,previous,2.0):
              # draw the bounding boxes
              draw_prediction(image,  round(x), round(y), round(x+w), round(y+h),(0, 255, 0))
            else:
              draw_prediction(image,  round(x), round(y), round(x+w), round(y+h),(0, 0, 255))
      previous = boxes
    result.write(image)
  result.release()


In [ ]:
with open(classesf, 'r') as f:
    classes = [line.strip() for line in f.readlines()]

In [ ]:
classes

['person',
 'bicycle',
 'car',
 'motorcycle',
 'airplane',
 'bus',
 'train',
 'truck',
 'boat',
 'traffic light',
 'fire hydrant',
 'stop sign',
 'parking meter',
 'bench',
 'bird',
 'cat',
 'dog',
 'horse',
 'sheep',
 'cow',
 'elephant',
 'bear',
 'zebra',
 'giraffe',
 'backpack',
 'umbrella',
 'handbag',
 'tie',
 'suitcase',
 'frisbee',
 'skis',
 'snowboard',
 'sports ball',
 'kite',
 'baseball bat',
 'baseball glove',
 'skateboard',
 'surfboard',
 'tennis racket',
 'bottle',
 'wine glass',
 'cup',
 'fork',
 'knife',
 'spoon',
 'bowl',
 'banana',
 'apple',
 'sandwich',
 'orange',
 'broccoli',
 'carrot',
 'hot dog',
 'pizza',
 'donut',
 'cake',
 'chair',
 'couch',
 'potted plant',
 'bed',
 'dining table',
 'toilet',
 'tv',
 'laptop',
 'mouse',
 'remote',
 'keyboard',
 'cell phone',
 'microwave',
 'oven',
 'toaster',
 'sink',
 'refrigerator',
 'book',
 'clock',
 'vase',
 'scissors',
 'teddy bear',
 'hair drier',
 'toothbrush']

In [ ]:
#COLORS = np.random.uniform(0, 255, size=(len(classes), 3))

net = cv2.dnn.readNet(weights, config)



In [ ]:
# Define the codec and create VideoWriter object.The output is stored in 'output.avi' file.

frame_width = int(cap.get(3)) 
frame_height = int(cap.get(4))

size = (frame_width, frame_height)
fcc = cv2.VideoWriter_fourcc(*'XVID')
result = cv2.VideoWriter(FILE_OUTPUT, fcc, 10, size)



In [ ]:
if __name__ == "__main__":
  video_read()
  

Streaming output truncated to the last 5000 lines.
[459.0, 34.0, 30, 24]
[402.0, 218.5, 76, 77]
[519.5, 150.5, 73, 109]
[732.0, 458.0, 104, 178]
[526.5, 92.5, 39, 47]
[557.0, 24.0, 24, 14]
[517.5, 63.0, 35, 26]
[510.5, 25.5, 25, 15]
[536.0, 3.0, 32, 12]
[461.0, 12.0, 28, 20]
[728.0, 363.0, 112, 114]
[545.5, 11.5, 27, 15]
[508.5, 18.0, 25, 14]
163 255
[312.5, 347.5, 125, 109]
[473.5, 383.0, 99, 134]
[510.0, 36.0, 30, 22]
[441.0, 121.5, 48, 39]
[353.5, 110.5, 57, 47]
[579.5, 46.5, 41, 31]
[444.0, 66.5, 40, 39]
[693.5, 202.5, 115, 133]
[399.0, 221.5, 82, 75]
[458.5, 32.5, 31, 25]
[519.0, 149.0, 76, 110]
[732.0, 458.5, 104, 177]
[518.0, 64.5, 36, 23]
[556.5, 23.0, 25, 14]
[512.5, 26.0, 25, 16]
[509.5, 17.5, 23, 13]
[730.5, 338.5, 105, 133]
[538.0, 3.5, 30, 11]
[526.5, 95.0, 39, 44]
[462.0, 11.5, 28, 19]
[532.0, 124.5, 40, 31]
[546.5, 11.5, 27, 15]
163 255
[310.0, 349.0, 124, 110]
[440.5, 122.0, 49, 40]
[510.0, 36.0, 30, 22]
[353.5, 110.5, 57, 47]
[579.0, 48.0, 42, 30]
[474.0, 386.0, 98, 13